In [ ]:
import torch
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["ALGICM_BACKEND"] = "torch"

In [ ]:
ckpt = torch.load("/data/sdv1/zhoutianqi/icm_tlx/icm/yolov5_s-v61_fast_1xb64-50e_voc_20221017_234156-0009b33e.pth")

In [ ]:
ckpt["state_dict"].keys()

In [ ]:
loss_cls_weight = 0.5
loss_bbox_weight = 0.05
loss_obj_weight = 1.0

# The scaling factor that controls the depth of the network structure
deepen_factor = 0.33
# The scaling factor that controls the width of the network structure
widen_factor = 0.5
num_classes = 1
num_det_layers = 3
img_scale = (640, 640)
norm_cfg = dict(type="BN2d", momentum=0.03, eps=0.001)
strides = [8, 16, 32]
anchors = [
    [(10, 13), (16, 30), (33, 23)],  # P3/8
    [(30, 61), (62, 45), (59, 119)],  # P4/16
    [(116, 90), (156, 198), (373, 326)],  # P5/32
]

model_test_cfg = dict(
    # The config of multi-label for multi-class prediction.
    multi_label=True,
    # The number of boxes before NMS
    nms_pre=2000,
    score_thr=0.1,  # Threshold to filter out boxes.
    nms=dict(type="nms", iou_threshold=0.1),  # NMS type and threshold
    max_per_img=300,
)  # Max number of detections of each image


model_cfg = dict(
    type="YOLODetector",
    data_preprocessor=dict(type="BaseDataProcessor",
                           batch_preprocess=[dict(type="Stack")]),
    # data_postprocessor=dict(type="BasePostProcessor"),
    backbone=dict(
        type="YOLOv5CSPDarknet",
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        norm_cfg=norm_cfg,
        act_cfg=dict(type="SiLU"),
    ),
    neck=dict(
        type="YOLOv5PAFPN",
        deepen_factor=deepen_factor,
        widen_factor=widen_factor,
        in_channels=[256, 512, 1024],
        out_channels=[256, 512, 1024],
        num_csp_blocks=3,
        norm_cfg=norm_cfg,
        act_cfg=dict(type="SiLU"),
    ),
    head=dict(
        type="YOLOv5Head",
        head_module=dict(
            type="YOLOv5HeadModule",
            num_classes=num_classes,
            in_channels=[256, 512, 1024],
            widen_factor=widen_factor,
            featmap_strides=strides,
            num_base_priors=3,
        ),
        prior_generator=dict(
            type="YOLOAnchorGenerator", base_sizes=anchors, strides=strides
        ),
        # scaled based on number of detection layers
        loss_cls=dict(
            type="CrossEntropyLoss",
            use_sigmoid=True,
            reduction="mean",
            loss_weight=loss_cls_weight ,
        ),
        loss_bbox=dict(
            type="IoULoss",
            iou_mode="ciou",
            bbox_format="xywh",
            eps=1e-7,
            reduction="mean",
            loss_weight=loss_bbox_weight ,
            return_iou=True,
        ),
        loss_obj=dict(
            type="CrossEntropyLoss",
            use_sigmoid=True,
            reduction="mean",
            loss_weight=loss_obj_weight
            ,
        ),
        prior_match_thr=4.0,
        obj_level_weights=[4.0, 1.0, 0.4],
    ),
    test_cfg=model_test_cfg,
)

In [ ]:
from algicm.registry.common import MODELS

In [ ]:
model = MODELS.build(model_cfg)

In [ ]:
import copy

In [ ]:
dict_ = ckpt['state_dict']
temp_dict = copy.deepcopy(dict_)
for i,j in temp_dict.items():
    if i.endswith("tracked") :
       del dict_[i]

dict_["head.priors_base_sizes"] = model.state_dict()["head.priors_base_sizes"]
dict_["head.grid_offset"] = model.state_dict()["head.grid_offset"]
dict_["head.prior_inds"] = model.state_dict()["head.prior_inds"]

dict_model = copy.deepcopy(model.state_dict())
dict_model.move_to_end("head.priors_base_sizes")
dict_model.move_to_end("head.grid_offset")
dict_model.move_to_end("head.prior_inds")

for k,v in zip(dict_model.keys(),dict_.keys()):
    dict_model[k] = dict_[v]

In [ ]:
dict_model

In [15]:
torch.save(dict_model,"pretrain.pth")